## Access Ollama Server by Openai() function

#### Reference
- Structured Outputs with Ollama: https://github.com/jxnl/instructor/blob/main/docs/examples/ollama.md
- Controlling Large Language Model Output with Pydantic: https://medium.com/@mattchinnock/controlling-large-language-model-output-with-pydantic-74b2af5e79d1

In [3]:
#!pip install pydantic instructor openai yfinance

In [37]:
from openai import OpenAI
from pydantic import BaseModel, Field
import yfinance as yf
import instructor

In [11]:
company = "Google"

class StockInfo(BaseModel):
    company: str = Field(..., description="Name of the company")
    ticker: str = Field(..., description="Ticker symbol of the company")

In [12]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": f"Return the company name and the ticker symbol of the {company}."
        }
    ],
    response_model=StockInfo,
    max_retries=5
)

In [13]:
stock = yf.Ticker(resp.ticker)
hist = stock.history(period="1d")
stock_price = hist['Close'].iloc[-1]
print(f"The stock price of the {resp.company} is {stock_price}. USD")

The stock price of the Alphabet Inc. is 175.360107421875. USD


---

In [57]:
from datetime import datetime

In [58]:
today = datetime.now().strftime("%Y-%m-%d, %A")
today

'2024-05-25, Saturday'

In [52]:
class test(BaseModel):
    date: str = Field(..., description="Date of booking", examples=["2-2-2024", "None", "check"])
    period: str = Field(..., description="Period of booking", examples=["13:00 - 14:00", "None", "check"])

In [56]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="Ollama"
    ),
    mode=instructor.Mode.JSON,
)

prompt = """
Instruction:
- Return the date and period based on the user input.
- You must follow below examples as reference to answer question.
- If user do not provide a clear date and period, return "None" in date and period.
- If user ask for available period or what period can book, return "check" in date and period.
- User are only to book 1 hour. If user ask for booking more than 1 hour, return "None"
----------------------------------------------------------------
### Example of without date and period
Qurestion: Any Booking available at 2/2/2024
return: {'date' = '2-2-2024', 'period'= 'None'}

### Example of irrelevant query
Qurestion: Good morning / Hi
return: {'date' = 'None', 'period' = 'None'}

### Example of time period conversion
Qurestion: I want to book at 1/6/2022 from 05:00 pm - 06:00 pm
return: {'date' = '1/6/2022', 'period' = '17:00 - 18:00'}
----------------------------------------------------------------
Query:
I want to book 1 day
"""

response = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    response_model=test,
    max_retries=5,
)

response


Instruction:
- Return the date and period based on the user input.
- You must follow below examples as reference to answer question.
- If user do not provide a clear date and period, return "None" in date and period.
- If user ask for available period, return "check" in date and period.
- User are only to book 1 hour. If user ask for booking more than 1 hour, return "None"
----------------------------------------------------------------
### Example of without date and period
Qurestion: Any Booking available at 2/2/2024
return: {'date' = '2-2-2024', 'period'= 'None'}

### Example of irrelevant query
Qurestion: Good morning / Hi
return: {'date' = 'None', 'period' = 'None'}

### Example of time period conversion
Qurestion: I want to book at 1/6/2022 from 05:00 pm - 06:00 pm
return: {'date' = '1/6/2022', 'period' = '17:00 - 18:00'}
----------------------------------------------------------------
Query:
I want to book 1 day
Hi nice to meet you


test(date='None', period='None')

---

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [7]:
llm = ChatOllama(model="llama3",
                 keep_alive=-1,
                 temperature=0,
                 max_new_tokens=512)

In [59]:
content = """
Today: 25/5/2024, Saturday

What's the date of next Wednesday
"""

messages = [
    HumanMessage(
        content=content
    )
]

chat_model_response = llm.invoke(messages)
print(chat_model_response.content)

Since today is Saturday (25th), next Wednesday would be:

28th May 2024


In [66]:
prompt = ChatPromptTemplate.from_template("Today: {date}\nThere is 31 days in May\nWhat's the date of next Wednesday")
chain = prompt | llm
print(chain.invoke({"date":"29/5/2024, Wednesday"}))

content='Since today is Wednesday (29th), the next Wednesday would be:\n\nWednesday, 6th June 2024' response_metadata={'model': 'llama3', 'created_at': '2024-05-24T17:44:17.871468Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1292868400, 'load_duration': 3909900, 'prompt_eval_count': 19, 'prompt_eval_duration': 350885000, 'eval_count': 24, 'eval_duration': 934218000} id='run-2b92c0a5-7727-4247-a6fc-44f6a0368a1f-0'
